In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from src.training.datasets import CustomImageFolder
import os
from settings import settings
import torchvision.models as models

/Users/kevywilly/felix-ai/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


********************************
*	Robot = felixMac
********************************

****************************************
*	Loading App Settings
****************************************



In [2]:
model = models.alexnet(pretrained=True)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 3)
model.classifier[6].in_features
model.train()

/Users/kevywilly/felix-ai/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kevywilly/felix-ai/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [3]:
images_path=os.path.join(settings.TRAINING.data_root, "training/ternary")
random_flip=True
target_flips={1: 2, 2: 1}

In [4]:
dataset = CustomImageFolder(
            root=images_path,
            target_flips=target_flips,
            transform=transforms.Compose([
                transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ],

            )
        )

In [5]:
batch_size = 8
test_pct = 20
test_size = int(len(dataset)*test_pct/100.0)

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - test_size, test_size])

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=128,
    shuffle=True,
    num_workers=0,
)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [7]:
n_epochs = 10

# Stuff to store
train_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)

for it in range(n_epochs):
  train_loss = []
  for inputs, targets in train_loader:
    # move data to GPU


    print(inputs.shape)
    # reshape the input
    #inputs = inputs.view(-1, 224,224)
    print(inputs.shape)

    # zero the parameter gradients
    optimizer.zero_grad()

    # Forward pass
    outputs = model.forward(inputs)
    print(outputs)
    loss = criterion(outputs, targets)

    # Backward and optimize
    loss.backward()
    optimizer.step()

    train_loss.append(loss.item())

  # Get train loss and test loss
  train_loss = np.mean(train_loss) # a little misleading

  test_loss = []
  for inputs, targets in test_loader:
    #inputs = inputs.view(-1, 224,224)
    outputs = model.forward(inputs)
    loss = criterion(outputs, targets)
    test_loss.append(loss.item())
  test_loss = np.mean(test_loss)

  # Save losses
  train_losses[it] = train_loss
  test_losses[it] = test_loss

  print(f'Epoch {it+1}/{n_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

torch.Size([8, 3, 224, 224])
torch.Size([8, 3, 224, 224])
tensor([[-0.5172,  0.5515,  1.1923],
        [-0.3540, -0.7053,  0.1443],
        [ 0.0201, -0.0664, -0.7616],
        [ 0.1726,  0.3159, -0.8366],
        [-0.8079,  1.0920, -0.2652],
        [-0.4647,  1.4174,  0.0277],
        [-0.7716, -0.5360, -0.1353],
        [-0.1718,  0.6487, -0.7189]], grad_fn=<AddmmBackward0>)
torch.Size([8, 3, 224, 224])
torch.Size([8, 3, 224, 224])
tensor([[ 1.0512,  0.0751,  1.2798],
        [ 0.6715,  0.6542,  1.3594],
        [-0.2765,  2.0656,  0.9120],
        [ 0.5728,  0.3853, -0.0873],
        [ 0.2543, -0.1882,  0.3223],
        [ 0.4907,  0.8765, -0.4155],
        [ 1.3118, -0.0622, -0.7604],
        [ 0.6387, -0.5942,  1.6479]], grad_fn=<AddmmBackward0>)
torch.Size([8, 3, 224, 224])
torch.Size([8, 3, 224, 224])
tensor([[-0.2760,  7.4486, -7.5615],
        [ 0.4238,  6.1977, -5.7597],
        [ 1.4301,  9.3256, -7.1679],
        [ 1.0277,  8.3750, -7.9123],
        [ 1.3044,  9.1237, -8.00

KeyboardInterrupt: 